In [ ]:
pip install fuzzywuzzy


In [ ]:
pip install antigranular

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.0/176.0 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 36.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 50.8 MB/s eta 0:00:00


In [ ]:
import antigranular as ag
session = ag.login(<client_id>,<client_secret>, competition = "Harvard OpenDP Hackathon")

Dataset "Flight Company Dataset" loaded to the kernel as flight_company_dataset

Dataset "Health Organisation Dataset" loaded to the kernel as health_organisation_dataset

Connected to Antigranular server session id: 901210c1-2172-4f28-8512-07ae289038ef, the session will time out if idle for 25 minutes
Cell magic '%%ag' registered successfully, use `%%ag` in a notebook cell to execute your python code on Antigranular private python server
🚀 Everything's set up and ready to roll!


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
%%ag
import pandas as pd
# Display the first few rows of the DataFrame




In [ ]:
%%ag


flight_company_dataset.dtypes

In [ ]:
# %%ag



In [ ]:
# %%ag

# import op_pandas
# op_pandas.to_datetime(
#         arg: PrivateSeries,
#         errors: str,
#         dayfirst: boolean,
#         yearfirst: boolean,
#         utc: boolean,
#         format: str,
#         exact: bool,
#         unit: str,
#         infer_datetime_format: bool,
#         origin: scaler,
#         cache: boolean,
# ):

In [ ]:
# %%ag
# import op_pandas
# combined=op_pandas.merge(flight_company_dataset, health_organisation_dataset, how="inner",left_on=['passenger_firstname', 'passenger_lastname','passenger_date_of_birth'], right_on=['patient_firstname','patient_lastname', 'patient_date_of_birth'])

In [ ]:
%%ag

import op_pandas
from fuzzywuzzy import fuzz

# Assuming flight_company_dataset and health_organisation_dataset are your DataFrames

# Function to calculate fuzzy match ratio for names
def fuzzy_match_ratio(name1, name2):
    return fuzz.ratio(name1, name2)

# Merge datasets based on fuzzy matching of names and exact matching of date of birth
combined = op_pandas.merge(flight_company_dataset, health_organisation_dataset, how="inner",
                           left_on='passenger_date_of_birth', right_on='patient_date_of_birth')

# Apply fuzzy matching on first and last names
combined['Name_Match_Score'] = combined.apply(lambda row: fuzzy_match_ratio(row['passenger_firstname'], row['patient_firstname']) +
                                                       fuzzy_match_ratio(row['passenger_lastname'], row['patient_lastname']), axis=1)

# Set a threshold for the fuzzy match score
threshold = 150  # You can adjust this threshold based on your requirements

# Filter the results based on the fuzzy match score and exact date of birth match
matched_records = combined[(combined['Name_Match_Score'] >= threshold) &
                            (combined['passenger_date_of_birth'] == combined['patient_date_of_birth'])]

# Drop the intermediate columns used for fuzzy matching
matched_records = matched_records.drop(['Name_Match_Score'], axis=1)

# Display the matched records
print(matched_records)


ImportError: Restricted import: fuzzywuzzy is not allowed


In [ ]:
%%ag

combined=combined.where(combined['covidtest_result']=="positive")
combined=combined.where(combined['passenger_firstname']==combined['patient_firstname'])
combined=combined.where(combined['passenger_lastname']==combined['patient_lastname'])
combined=combined.where(combined['passenger_date_of_birth']==combined['patient_date_of_birth'])



In [ ]:
%%ag

ag_print("Columns: \n",combined.columns)
ag_print("Metadata: \n", combined.metadata)
ag_print("Dtypes: \n", combined.dtypes)
ag_print("Describe:\n", combined.describe(eps=1))

In [ ]:
# %%ag
# import syft as sy

# hook = sy.TorchHook(torch)
# # Assuming combined is a PrivateDataset in PySyft

# # Define a function to convert date strings to datetime objects
# def convert_to_datetime(date_str):
#     return pd.to_datetime(date_str, errors='coerce')

# # Use the map method to apply the function to the PrivateDataset
# combined['covidtest_date'] = combined['covidtest_date'].map(convert_to_datetime)

In [ ]:
import syft as sy

hook = sy.TorchHook(torch)
# Assuming combined is a PrivateDataset in PySyft

# Define a function to convert date strings to datetime objects
def convert_to_datetime(date_str):
    return pd.to_datetime(date_str, errors='coerce')

# Use the map method to apply the function to the PrivateDataset
combined['date_column'] = combined['date_column'].map(convert_to_datetime)

In [ ]:
%%ag

# def compare.custom(cmp: func(x,y) -> int , left_col , right_col , label)

# Using a custom compare rule.

from datetime import datetime

def cmp(combined):
    # combined['covidtest_date']=datetime.strftime(combined['covidtest_date'], "%Y-%m-%d")
    # combined['flight_date']=datetime.strftime(combined['flight_date'], "%Y-%m-%d")


    combined['covidtest_date'] = combined['covidtest_date'].apply(lambda x: pd.to_datetime(x, errors='coerce'))
    combined['flight_date'] = combined['flight_date'].apply(lambda x: pd.to_datetime(x, errors='coerce'))


    combined['temp']=(combined['covidtest_date']-combined['flight_date']).days
    combined['var']=0;
    if(combined['temp']>15):
      combined['var']=0;
    # Check if the dates are within two weeks (14 days) apart
    else :
      combined['var']=1;

    return combined;

temp_check=cmp(combined);
# compare.custom(cmp,"flight_date","covidtest_date",label="date_cmp")

# def op_recordlinkage.Compare.compute(
#     pairs:op_recordlinkage.Index.compute() ,
#     x : PrivateDataFrame ,
#     x_link: PrivateDataFrame
# ) -> PrivateDataFrame




In [ ]:
# %%ag

# import op_splink.duckdb.comparison_template_library as ctl
# from op_splink.duckdb.blocking_rule_library import block_on

# date_difference = ctl.date_comparison("date", cast_strings_to_date = True, include_exact_match_level = False, damerau_levenshtein_thresholds = [], datediff_thresholds = [14], datediff_metrics = ["day"])

# ag_print(date_difference.human_readable_description)

In [ ]:
# %%ag

# import recordlinkage as rl
# from recordlinkage.compare import Exact, String, Numeric, Date


# from datetime import datetime


# class CompareDates(BaseCompareFeature):
#     def cmp(date_str1 , date_str2):
#       # Convert date strings to datetime objects
#       date1 = datetime.strptime(date_str1, "%Y-%m-%d")
#       date2 = datetime.strptime(date_str2, "%Y-%m-%d")

#       # Calculate the absolute difference in days
#       days_apart = (date2 - date1).days
#       # Check if the dates are within two weeks (14 days) apart
#       if days_apart > 15:
#           return 0;
#       else:
#           return 1

# comparer = rl.Compare()

# comparer.add(CompareZipCodes('flight_date', 'covidtest_date', label='y_dates'))
# comparer.compute(pairs,flight_company_dataset , health_organisation_dataset)






In [ ]:
# session.submit_predictions(sub)


# result = ag_client.submit_predictions(sub)
# print(result)



In [ ]:
# %%ag

# compare = op_recordlinkage.Compare()

# def compare.custom(cmp: func(x,y) -> int , left_col , right_col , label)

# # Using a custom compare rule.
# from datetime import datetime
# def cmp(date_str1 , date_str2):
#     # Convert date strings to datetime objects
#     date1 = datetime.strptime(date_str1, "%Y-%m-%d")
#     date2 = datetime.strptime(date_str2, "%Y-%m-%d")

#     # Calculate the absolute difference in days
#     days_apart = (date2 - date1).days
#     # Check if the dates are within two weeks (14 days) apart
#     if days_apart <= 14:
#         return 1
#     else:
#         return 0


# compare.custom(cmp,"flight_date","covidtest_date",label="date_cmp")

In [ ]:
cons=health_organisation_dataset['covidtest_result']==
selected_rows=health_organisation_dataset[cons]

In [ ]:
%%ag
from dateutil import parser

# Function to standardize date formats
def standardize_date(date_str):
    try:
        # Try parsing the date using dateutil.parser
        return parser.parse(date_str).strftime('%Y-%m-%d')
    except ValueError:
        # If parsing fails, return a default value or handle accordingly
        return None

# Apply the standardize_date function to the 'dates' column
flight_company_dataset['fight_date'] = flight_company_dataset['flight_date'].apply(standardize_date)
